In [1]:

import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Spider
import scrapy
from scrapy import Request

class craiglistSpider(scrapy.Spider):
    name = "jobs"
    allowed_domains = ["craigslist.org"]
    start_urls = ["https://honolulu.craigslist.org/search/sss?query=surfboard&sort=rel"]

    def parse(self, response):
        jobs = response.xpath('//p[@class="result-info"]')

        for job in jobs:
            
            relative_url = job.xpath('a/@href').extract_first()
            absolute_url = response.urljoin(relative_url)
            
            yield Request(absolute_url, callback=self.parse_page, meta={'URL': absolute_url})
            
        relative_next_url = response.xpath('//a[@class="button next"]/@href').extract_first()
        absolute_next_url = "https://honolulu.craigslist.org" + relative_next_url

        yield Request(absolute_next_url, callback=self.parse)
        
    def parse_page(self, response):
        url = response.meta.get('URL')

        yield {
        'url': url,
        'title': response.xpath('//span[@id="titletextonly"]/text()').extract(),
        'price': response.xpath('//span[@class="price"]/text()').extract(),
        'description': "".join(line for line in response.xpath('//*[@id="postingbody"]/text()').extract()),
        'condition': response.xpath('//p[@class="attrgroup"]/span[1]/b/text()').extract(),    
        'manufacturer': response.xpath('//p[@class="attrgroup"]/span[2]/b/text()').extract(),
        'model_name': response.xpath('//p[@class="attrgroup"]/span[3]/b/text()').extract(),
        'size_dimensions': response.xpath('//p[@class="attrgroup"]/span[4]/b/text()').extract()
        }
process = CrawlerProcess({
'FEED_FORMAT': 'json',
'FEED_URI': 'surfboard.json',
'ROBOTSTXT_OBEY': False,
'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
'AUTOTHROTTLE_ENABLED': True,
'HTTPCACHE_ENABLED': True,
'LOG_ENABLED': False
})
                                         

process.crawl(craiglistSpider)
process.start()
